In [1]:
!python -m pip install -U transformers accelerate jiwer librosa soundfile


In [2]:
import pandas as pd
from pathlib import Path
import re, unicodedata
from jiwer import wer as jiwer_wer

BASE = Path(r"C:\Users\mohab\OneDrive\Desktop\NLP")
CLIPS = BASE / "clips"

train_df = pd.read_csv(BASE/"train.tsv", sep="\t")
dev_df   = pd.read_csv(BASE/"dev.tsv", sep="\t")
test_df  = pd.read_csv(BASE/"test.tsv", sep="\t")

def normalize_es(text: str) -> str:
    text = str(text).lower().strip()
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r"[^a-záéíóúüñ\s]", " ", text, flags=re.UNICODE)
    text = re.sub(r"\s+", " ", text).strip()
    return text

print("Loaded:", len(train_df), len(dev_df), len(test_df))
print("Example:", train_df.loc[0, ["path","sentence"]].to_dict())


Loaded: 196006 15338 15338
Example: {'path': 'common_voice_es_22140775.mp3', 'sentence': 'Esto le llevó a Holanda, Inglaterra, Francia e Italia.'}


In [3]:
from transformers import pipeline

asr_w2v = pipeline(
    task="automatic-speech-recognition",
    model="facebook/wav2vec2-large-960h-lv60-self",
    device=-1  # CPU
)

print("Wav2Vec2 pipeline ready.")


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Wav2Vec2 pipeline ready.


In [4]:
import shutil
print(shutil.which("ffmpeg"))


C:\Users\mohab\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-8.0.1-full_build\bin\ffmpeg.EXE


In [5]:
import shutil
print(shutil.which("ffmpeg"))


C:\Users\mohab\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-8.0.1-full_build\bin\ffmpeg.EXE


In [9]:
import shutil
print(shutil.which("ffmpeg"))


None


In [1]:
import shutil
print(shutil.which("ffmpeg"))

C:\Users\mohab\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-8.0.1-full_build\bin\ffmpeg.EXE


In [2]:
def w2v_transcribe(path):
    audio_path = str(CLIPS / path)
    out = asr_w2v(audio_path)
    return out["text"]

def evaluate_w2v(df, limit=50):
    wers = []
    for _, row in df.head(limit).iterrows():
        ref = normalize_es(row["sentence"])
        hyp = normalize_es(w2v_transcribe(row["path"]))
        wers.append(jiwer_wer(ref, hyp))
    return sum(wers) / len(wers)

w2v_dev_wer = evaluate_w2v(dev_df, limit=20)  # start with 20
print("Wav2Vec2 Dev WER (first 20):", w2v_dev_wer)


NameError: name 'dev_df' is not defined

In [3]:
import pandas as pd
from pathlib import Path

BASE = Path(r"C:\Users\mohab\OneDrive\Desktop\NLP")
CLIPS = BASE / "clips"

train_df = pd.read_csv(BASE / "train.tsv", sep="\t")
dev_df   = pd.read_csv(BASE / "dev.tsv", sep="\t")
test_df  = pd.read_csv(BASE / "test.tsv", sep="\t")

print(len(train_df), len(dev_df), len(test_df))


196006 15338 15338


In [4]:
import re, unicodedata

def normalize_es(text):
    text = str(text).lower().strip()
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r"[^a-záéíóúñü ]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [5]:
print(normalize_es(dev_df["sentence"].iloc[0]))


estaba rodeada de jardines y de un prado arbolado


In [6]:
from jiwer import wer as jiwer_wer


In [7]:
w2v_dev_wer = evaluate_w2v(dev_df, limit=20)
print("Wav2Vec2 Dev WER (first 20):", w2v_dev_wer)


NameError: name 'asr_w2v' is not defined

In [9]:
import torch
from transformers import pipeline
from jiwer import wer as jiwer_wer


In [10]:
device = 0 if torch.cuda.is_available() else -1

asr_w2v = pipeline(
    "automatic-speech-recognition",
    model="facebook/wav2vec2-large-960h",
    device=device
)


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

C:\Users\mohab\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mohab\.cache\huggingface\hub\models--facebook--wav2vec2-large-960h. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fall

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

Device set to use cpu


In [6]:
device = 0 if torch.cuda.is_available() else -1

asr_w2v = pipeline(
    "automatic-speech-recognition",
    model="facebook/wav2vec2-large-960h",
    device=device
)

NameError: name 'torch' is not defined

In [7]:
import torch
from transformers import pipeline
from jiwer import wer as jiwer_wer


In [8]:
device = 0 if torch.cuda.is_available() else -1

asr_w2v = pipeline(
    "automatic-speech-recognition",
    model="facebook/wav2vec2-large-960h",
    device=device
)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [9]:
def w2v_transcribe(path):
    audio_path = str(CLIPS / path)
    out = asr_w2v(audio_path)
    return out["text"]


In [10]:
def evaluate_w2v(df, limit=10):
    wers = []
    for _, row in df.head(limit).iterrows():
        ref = normalize_es(row["sentence"])
        hyp = normalize_es(w2v_transcribe(row["path"]))
        wers.append(jiwer_wer(ref, hyp))
    return sum(wers) / len(wers)


In [11]:
w2v_dev_wer = evaluate_w2v(dev_df, limit=10)
print("Wav2Vec2 Dev WER (first 10):", w2v_dev_wer)


Wav2Vec2 Dev WER (first 10): 1.1026190476190476


In [12]:
import sys
print(sys.executable)


C:\Users\mohab\AppData\Local\Programs\Python\Python313\python.exe


In [13]:
!{sys.executable} -m pip install -U openai-whisper jiwer


     ---------------------------------------- 0.0/803.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/803.2 kB ? eta -:--:--
     ------------- -------------------------- 262.1/803.2 kB ? eta -:--:--
     ------------------------ ------------- 524.3/803.2 kB 1.3 MB/s eta 0:00:01
     ------------------------ ------------- 524.3/803.2 kB 1.3 MB/s eta 0:00:01
     --------------------------------------- 803.2/803.2 kB 701.0 kB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/879.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/879.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/879.1 kB ? eta -:--

In [14]:
import shutil
print("ffmpeg:", shutil.which("ffmpeg"))
print("ffprobe:", shutil.which("ffprobe"))


ffmpeg: C:\Users\mohab\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-8.0.1-full_build\bin\ffmpeg.EXE
ffprobe: C:\Users\mohab\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-8.0.1-full_build\bin\ffprobe.EXE


In [15]:
import whisper
from jiwer import wer as jiwer_wer

# pick a small model first (faster): tiny, base, small...
WHISPER_MODEL_NAME = "tiny"  # change to "base" later if you want better WER

whisper_model = whisper.load_model(WHISPER_MODEL_NAME)

def whisper_transcribe(path):
    audio_path = str(CLIPS / path)
    result = whisper_model.transcribe(audio_path, language="es", fp16=False)  # fp16=False for CPU/Windows
    return result["text"]

def evaluate_whisper(df, limit=20):
    wers = []
    for _, row in df.head(limit).iterrows():
        ref = normalize_es(row["sentence"])
        hyp = normalize_es(whisper_transcribe(row["path"]))
        wers.append(jiwer_wer(ref, hyp))
    return sum(wers) / len(wers)

whisper_dev_wer = evaluate_whisper(dev_df, limit=20)
print(f"Whisper ({WHISPER_MODEL_NAME}) Dev WER (first 20):", whisper_dev_wer)


100%|█████████████████████████████████████| 72.1M/72.1M [00:11<00:00, 6.53MiB/s]


Whisper (tiny) Dev WER (first 20): 0.5022907647907647
